In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('data/data_imputation_train.csv')
val_df = pd.read_csv('data/data_imputation_val.csv')
test_df = pd.read_csv('data/data_imputation_test.csv')
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,...,emp_length_was_missing,tot_coll_amt_was_missing,total_rev_hi_lim_was_missing,mo_sin_old_rev_tl_op_is_missing,tot_cur_bal_is_missing,bc_util_is_missing,avg_cur_bal_is_missing,bc_open_to_buy_is_missing,is_thin_file,has_modern_metrics
0,3600.0,36 months,13.99,123.03,C,C4,10+ years,MORTGAGE,55000.0,Not Verified,...,0,0,0,0,0,0,0,0,0,1
1,24700.0,36 months,11.99,820.28,C,C1,10+ years,MORTGAGE,65000.0,Not Verified,...,0,0,0,0,0,0,0,0,0,1
2,20000.0,60 months,10.78,432.66,B,B4,10+ years,MORTGAGE,63000.0,Not Verified,...,0,0,0,0,0,0,0,0,0,1
3,10400.0,60 months,22.45,289.91,F,F1,3 years,MORTGAGE,104433.0,Source Verified,...,0,0,0,0,0,0,0,0,0,1
4,11950.0,36 months,13.44,405.18,C,C3,4 years,RENT,34000.0,Source Verified,...,0,0,0,0,0,0,0,0,0,1


In [5]:
df['emp_length']

0          10+ years
1          10+ years
2          10+ years
3            3 years
4            4 years
             ...    
1119706      5 years
1119707      9 years
1119708      3 years
1119709    10+ years
1119710      6 years
Name: emp_length, Length: 1119711, dtype: object

In [6]:
def convert_emp_length(df):
    mapping = {
        '10+ years': 10,
        '9 years': 9,
        '8 years': 8,
        '7 years': 7,
        '6 years': 6,
        '5 years': 5,
        '4 years': 4,
        '3 years': 3,
        '2 years': 2,
        '1 year': 1,
        '< 1 year': 0
    }
    df['emp_length_num'] = df['emp_length'].map(mapping).fillna(0).astype(int)
    df = df.drop(columns=['emp_length'])
    return df

df = convert_emp_length(df)
val_df = convert_emp_length(val_df)
test_df = convert_emp_length(test_df)

In [7]:
# Adding features created in feature engineering file
df['rate_premium'] = df['int_rate'] - df['fed_funds_rate']
df['emp_unemployment_risk'] = (11 - df['emp_length_num']) * df['unemployment_rate']
df['rev_bal_to_inc'] = df['revol_bal'] / (df['annual_inc'] + 1)

val_df['rate_premium'] = val_df['int_rate'] - val_df['fed_funds_rate']
val_df['emp_unemployment_risk'] = (11 - val_df['emp_length_num']) * val_df['unemployment_rate']
val_df['rev_bal_to_inc'] = val_df['revol_bal'] / (val_df['annual_inc'] + 1)

test_df['rate_premium'] = test_df['int_rate'] - test_df['fed_funds_rate']
test_df['emp_unemployment_risk'] = (11 - test_df['emp_length_num']) * test_df['unemployment_rate']
test_df['rev_bal_to_inc'] = test_df['revol_bal'] / (test_df['annual_inc'] + 1)

In [8]:
def correlations(df, target_col='target'):
    num_col = df.select_dtypes(include=[np.number]).columns.to_list()    
    corr_matrix = df[num_col].corr()     
    target_corr = corr_matrix[target_col].drop(target_col).sort_values(ascending=False)

    print('Features Correlation with Target')
    print(target_corr)

    high_target_corr = target_corr[abs(target_corr) > 0.1].index.tolist()
    print('\nHighly correlated features with Target')
    for f in high_target_corr:
        print(f'- {f}: {target_corr[f]:.3f}')

    multi_pair = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.8:
                multi_pair.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
    
    if multi_pair:
        print('\nHigh multicollinearity')
        for feat1, feat2, corr_val in multi_pair:
            print(f"  - {feat1} <-> {feat2}: {corr_val:.3f}")
    else:
        print('\nNo severe multicollinearity detected')

    return target_corr, high_target_corr, multi_pair

In [9]:
target_corr, high_corr, multi_pair = correlations(df)

Features Correlation with Target
int_rate                0.260750
rate_premium            0.258895
acc_open_past_24mths    0.115683
dti                     0.105004
num_tl_op_past_12m      0.100465
                          ...   
fico_range_high        -0.130251
fico_range_low         -0.130252
last_fico_range_low    -0.559699
last_fico_range_high   -0.646366
policy_code                  NaN
Name: target, Length: 104, dtype: float64

Highly correlated features with Target
- int_rate: 0.261
- rate_premium: 0.259
- acc_open_past_24mths: 0.116
- dti: 0.105
- num_tl_op_past_12m: 0.100
- fico_range_high: -0.130
- fico_range_low: -0.130
- last_fico_range_low: -0.560
- last_fico_range_high: -0.646

High multicollinearity
  - loan_amnt <-> installment: 0.954
  - int_rate <-> rate_premium: 0.999
  - fico_range_low <-> fico_range_high: 1.000
  - mths_since_last_record <-> has_record: -1.000
  - open_acc <-> num_sats: 0.921
  - revol_bal <-> total_rev_hi_lim: 0.809
  - revol_util <-> bc_util: 0.

**Columns to Drop before training**
- installment
- fico_range_high
- num_sats
- last_fico_range_low & last_fico_range_high (these are updated monthly after loan is granted)
- out_prncp & out_prncp_inv: Used to track active loans
- policy_code (NaN)
- Thin file grp (tot_coll_amt_was_missing, total_rev_hi_lim_was_missing, mo_sin_old_rev_tl_op_is_missing, tot_cur_bal_is_missing, avg_cur_bal_is_missing, mo_sin_rcnt_rev_tl_op)
- History flags (has_record, has_dlq, has_derog)
- tot_cur_bal
- bc_util
- num_rev_tl_bal_gt_0
- mo_sin_rcnt_tl
- num_rev_tl_bal_gt_0
- num_bc_tl
- revol_bal

In [10]:
final_drop = [
    'last_fico_range_high', 'last_fico_range_low', 'out_prncp', 'out_prncp_inv', 'num_bc_tl', 'revol_bal', 'inflation_rate',
    'installment', 'fico_range_high', 'num_sats', 'tot_cur_bal', 'bc_util', 'num_rev_tl_bal_gt_0', 'bc_open_to_buy_is_missing',
    'policy_code', 'has_record', 'has_dlq', 'has_derog', 'mths_since_recent_revol_delinq', 'num_op_rev_tl', 'total_il_high_credit_limit',
    'tot_coll_amt_was_missing', 'total_rev_hi_lim_was_missing', 'avg_cur_bal_is_missing', 'mo_sin_rcnt_rev_tl_op', 'bc_util_is_missing',
    'mo_sin_old_rev_tl_op_is_missing', 'tot_cur_bal_is_missing', 'num_rev_tl_bal_gt_0','mo_sin_rcnt_tl', 'has_installment_history',
    'bc_open_to_buy', 'num_actv_bc_tl', 'num_bc_sats', 'has_delinq', 'rate_premium', 'emp_length_num',
    
]

X_train = df.drop(columns=final_drop)
X_val = val_df.drop(columns=final_drop)
X_test = test_df.drop(columns=final_drop)

In [11]:
X_train.columns

Index(['loan_amnt', 'term', 'int_rate', 'grade', 'sub_grade', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'purpose', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_util', 'total_acc', 'initial_list_status',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'application_type', 'acc_now_delinq', 'tot_coll_amt', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'chargeoff_within_12_mths',
       'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mort_acc',
       'mths_since_recent_bc', 'mths_since_recent_

In [12]:
print(X_train['issue_d'][0])
print(X_train['earliest_cr_line'][0])

2015-12-01
Aug-2003


In [13]:
# Calculating credit age
def calculate_credit_age(df):
    issue_date = pd.to_datetime(df['issue_d'], format = "%Y-%m-%d")
    earliest_cr = pd.to_datetime(df['earliest_cr_line'], format = "%b-%Y")

    return ((issue_date.dt.year - earliest_cr.dt.year) * 12 + (issue_date.dt.month - earliest_cr.dt.month))

X_train['credit_hist_age_mths'] = calculate_credit_age(X_train)
X_val['credit_hist_age_mths'] = calculate_credit_age(X_val)
X_test['credit_hist_age_mths'] = calculate_credit_age(X_test)

In [14]:
X_train['disbursement_method'].value_counts()

disbursement_method
Cash         1117550
DirectPay       2161
Name: count, dtype: int64

In [15]:
X_train['pymnt_plan'].value_counts()

pymnt_plan
n    1119711
Name: count, dtype: int64

In [16]:
y_train = X_train['target']
y_val = X_val['target']
y_test = X_test['target']

to_drop = ['target', 'loan_status', 'issue_d', 'month_year', 'earliest_cr_line', 'grade', 'pymnt_plan', 'disbursement_method']
X_train.drop(columns = to_drop, inplace = True)
X_val.drop(columns = to_drop, inplace = True)
X_test.drop(columns = to_drop, inplace = True)

In [18]:
X_train.shape

(1119711, 77)

In [19]:
train_df = pd.concat([X_train, y_train], axis=1)
train_df.to_csv('data/data_imputation_train.csv', index=False)

val_df = pd.concat([X_val, y_val], axis=1)
val_df.to_csv('data/data_imputation_val.csv', index=False)

test_df = pd.concat([X_test, y_test], axis=1)
test_df.to_csv('data/data_imputation_test.csv', index=False)